# Importing all libraries

In [ ]:
import io, os, re
#os.environ['GOOGLE_APPLICATION_CREDENTIALS']= '/Users/dhirajhasija/opt/anaconda3/vision.json'
#from Levenshtein import distance as lev
#from google.cloud import vision
#from google.cloud.vision_v1 import types
#from google.cloud.vision_v1 import AnnotateImageResponse
import json
import numpy as np
import pandas as pd
import argparse
from enum import Enum
import cv2

#from google.cloud import vision
from PIL import Image, ImageDraw, ImageOps
#client = vision.ImageAnnotatorClient()
pd.set_option('display.max_rows', None)
import fastwer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

In [ ]:
client = vision.ImageAnnotatorClient()

In [15]:
!pip install fastwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached fastwer-0.1.3.tar.gz (4.6 kB)
  Using cached fastwer-0.1.tar.gz (3.6 kB)
ERROR: Could not find a version that satisfies the requirement fastwer (from versions: 0.1, 0.1.3)
ERROR: No matching distribution found for fastwer


In [ ]:
client = vision.ImageAnnotatorClient()
    # [END vision_python_migration_client]

    # The name of the image file to annotate
file_name = os.path.abspath('Fontfre_Clean_TE.png')

    # Loads the image into memory
with io.open(file_name, 'rb') as image_file:
    content = image_file.read()

image = vision.Image(content=content)
    
#response = client.document_text_detection(image=image)
#texts = response.full_text_annotation



In [ ]:
response = client.document_text_detection(image=image)

# Writing detected words adjacent to image

In [ ]:
#Writing words adjacent to image 


from PIL import ImageFont

def draw_boxes(image, bounds, color, words):
    draw = ImageDraw.Draw(image)
    
    for i in range(0,len(words)):
            draw.text((bounds[i].vertices[0].x-1000,bounds[i].vertices[0].y), words[i],font=ImageFont.truetype('/Library/Fonts/Arial.ttf', 15), align ="centre", strokewidth = 2, fill ="white") 

    for bound in bounds:
        draw.polygon(
            [
                bound.vertices[0].x-1000,
                bound.vertices[0].y,
                bound.vertices[1].x-1000,
                bound.vertices[1].y,
                bound.vertices[2].x-1000,
                bound.vertices[2].y,
                bound.vertices[3].x-1000,
                bound.vertices[3].y,
            ], None, color,
        )
            
    return image

def get_document_bounds(document):
    bounds = []
    words = []
    for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    bounds.append(word.bounding_box)
                    word_text = ''.join([symbol.text for symbol in word.symbols])
                    words.append(word_text)

    return bounds, words

def render_doc_text(filein, fileout, document):
    image = Image.open(filein)
    image = image.rotate(270)
    bounds, words = get_document_bounds(document)
    draw_boxes(image, bounds, "yellow", words)

    if fileout != 0:
        image.save(fileout)
        
        
        
def image_to_byte_array(image:Image):
    imgByteArr = io.BytesIO()
    image.save(imgByteArr, format='JPEG')
    imgByteArr = imgByteArr.getvalue()
    return imgByteArr


directory = 'Improvement tests'

for folder in os.listdir(directory):
    if folder in ['Bounded + text']:
        folderpath = os.path.join(directory, folder)
        for folder1 in os.listdir(folderpath):
            if folder1 in ['Restoration']:
                folderpath1 = os.path.join(folderpath, folder1)
                if os.path.isdir(folderpath1) == True:
                    for imagename in os.listdir(folderpath1):
                        if imagename != '.DS_Store':   
                            imagepath = os.path.join(folderpath1, imagename)
                            imagepath = os.path.abspath(imagepath)
                            image = ImageOps.exif_transpose(Image.open(imagepath))
                            image = image.rotate(270)
                            content = image_to_byte_array(image)
                            image = vision.Image(content=content)
                            response = client.document_text_detection(image=image)  
                            document = response.full_text_annotation
                            render_doc_text(imagepath,imagepath.split('.')[0] + '-words.jpg', document)

# Drawing bounding boxes and creating output dataframe

In [ ]:
Distortion_map = {
    'S':'Single',
    'M':'Multiple'
}

Light_condition_map = {'1':'Day','2':'Day + Cieling','3':'Night + Table', '4' : 'Table + Shadow', '5': 'Table + Grid Shadow'}


Blur_map = {'SMb1' : 'Horizontal', 'SMb2':'2D', 'MMb1' : 'Horizontal', 'MMb2':'2D','SOb1': 'f_22.5', 'SOb2' : 'f_21.5', 'SOb3': 'f_20.5', 'SOb4':'f_19.5', 'MOb1': 'f_21.66', 'MOb2':'f_19.66'}

def draw_boxes(image, bounds, color):
    draw = ImageDraw.Draw(image)

    for bound in bounds:
        draw.polygon(
            [
                bound.vertices[0].x,
                bound.vertices[0].y,
                bound.vertices[1].x,
                bound.vertices[1].y,
                bound.vertices[2].x,
                bound.vertices[2].y,
                bound.vertices[3].x,
                bound.vertices[3].y,
            ],
            None,
            color,
        )
        
        
    return image

def get_document_bounds(document, feature):
    bounds = []
    for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    for symbol in word.symbols:
                        if feature == FeatureType.SYMBOL:
                        
                            bounds.append(symbol.bounding_box)

                    if feature == FeatureType.WORD:
                        bounds.append(word.bounding_box)

                if feature == FeatureType.PARA:
                    bounds.append(paragraph.bounding_box)

            if feature == FeatureType.BLOCK:
                bounds.append(block.bounding_box)

    return bounds

def render_doc_text(filein, fileout, document):
    image = Image.open(filein)
    bounds = get_document_bounds(document, FeatureType.BLOCK)
    draw_boxes(image, bounds, "blue")
    bounds = get_document_bounds(document, FeatureType.PARA)
    draw_boxes(image, bounds, "red")
    bounds = get_document_bounds(document, FeatureType.WORD)
    draw_boxes(image, bounds, "yellow")

    if fileout != 0:
        image.save(fileout)
        
class FeatureType(Enum):
    PAGE = 1
    BLOCK = 2
    PARA = 3
    WORD = 4
    SYMBOL = 5

In [ ]:
#For smart Doc improved images
try:
    del df_comparison
except:
    print('df_comparison does not exist')
    
df_comparison = pd.DataFrame(columns= ['Imagename','Distortion', 'Doc_no', 'Light_condition', 'Long_angle','Lateral_angle','Type_of_blur','Type_of_sharpening','no_of_bounding_boxes', 'words','confidence'])

def image_to_byte_array(image:Image):
    imgByteArr = io.BytesIO()
    image.save(imgByteArr, format='JPEG')
    imgByteArr = imgByteArr.getvalue()
    return imgByteArr

# Name the directory containing image folders
directory = 'Improvement tests'

for folder in os.listdir(directory):
#Enter name of folders to compare; filename is folder name
    if folder in ['Sample']:
        folderpath = os.path.join(directory, folder)
        #for folder1 in os.listdir(folderpath):
        #    if folder1 not in ['.Ds_Store']:
        #        folderpath1 = os.path.join(folderpath, folder1)
        #        if os.path.isdir(folderpath1) == True:
        for imagename in os.listdir(folderpath):
            if imagename != '.DS_Store':   
                imagepath = os.path.join(folderpath, imagename)
                imagepath = os.path.abspath(imagepath)
                image = ImageOps.exif_transpose(Image.open(imagepath))
                
                content = image_to_byte_array(image)
                image = vision.Image(content=content)
                response = client.document_text_detection(image=image)
                
                document = response.full_text_annotation
                
                render_doc_text(imagepath,imagepath.split('.')[0] + 'bounded.jpg', document)

                for page in document.pages:
                    df_sect = pd.DataFrame(columns=['word','confidence'])
                    for block in page.blocks:
                        for paragraph in block.paragraphs:
                            for word in paragraph.words:

                                word_text = ''.join([
                                        symbol.text for symbol in word.symbols
                                    ])


                                df_sect =df_sect.append(
                                dict(
                                    word = word_text,
                                    confidence = word.confidence
                                ), ignore_index = True
                                )

                    print(imagename)

                    if len(imagename.split('_')) == 9:
                        df_comparison = df_comparison.append(
                            dict( 
                                Imagename = imagename.split('.')[0],
                                Distortion = Distortion_map[imagename.split('_')[0]],
                                Doc_no = imagename.split('_')[3][1:],
                                Light_condition = Light_condition_map[imagename.split('_')[4][1:]],
                                Long_angle = imagename.split('_')[7][1:-4],
                                Lateral_angle = imagename.split('_')[6][1:],
                                Type_of_blur = Blur_map[(imagename.split('_')[0] +imagename.split('_')[8][:-4])],
                                Type_of_sharpening = 'NA',
                                no_of_bounding_boxes = len(df_sect),
                                words = df_sect['word'].tolist() ,
                                confidence = df_sect['confidence'].mean(),
                                confidence_med = df_sect['confidence'].median()
                            ), ignore_index= True
                        )

                    if len(imagename.split('_')) == 2:
                        df_comparison = df_comparison.append(
                            dict( 
                                Imagename = imagename.split('.')[0],
                                Distortion = 'NA',
                                Doc_no = imagename.split('_')[1][:-4],
                                Light_condition = 'NA',
                                Long_angle = 'NA',
                                Lateral_angle = 'NA',
                                Type_of_blur = 'NA',
                                Type_of_sharpening = 'NA',
                                no_of_bounding_boxes = len(df_sect),
                                words = df_sect['word'].tolist() ,
                                confidence = df_sect['confidence'].mean(),
                                confidence_med = df_sect['confidence'].median()
                            ), ignore_index= True
                        )

                    if len(imagename.split('_')) == 10:
                        df_comparison = df_comparison.append(
                            dict( 
                                Imagename = imagename.split('.')[0],
                                Distortion = Distortion_map[imagename.split('_')[0]],
                                Doc_no = imagename.split('_')[3][1:],
                                Light_condition = Light_condition_map[imagename.split('_')[4][1:]],
                                Long_angle = imagename.split('_')[7][1:],
                                Lateral_angle = imagename.split('_')[6][1:],
                                Type_of_blur = Blur_map[(imagename.split('_')[0] +imagename.split('_')[8])],
                                Type_of_sharpening = (imagename.split('_')[9]).split('.')[0],
                                no_of_bounding_boxes = len(df_sect),
                                words = df_sect['word'].tolist() ,
                                confidence = df_sect['confidence'].mean(),
                                confidence_med = df_sect['confidence'].median()
                            ), ignore_index= True
                        )

In [ ]:
#Creating a count_vectorizer method
count_vectorizer = CountVectorizer(stop_words='english')
count_vectorizer = CountVectorizer()

In [ ]:
a = open('/Users/dhirajhasija/Downloads/OCR project/Ground Truth/page_30.txt', "r")
page_30 = (a.read())

a = open('/Users/dhirajhasija/Downloads/OCR project/Ground Truth/page_29.txt', "r")
page_29 = (a.read())

a = open('/Users/dhirajhasija/Downloads/OCR project/Ground Truth/page_28.txt', "r")
page_28 = (a.read())

a = open('/Users/dhirajhasija/Downloads/OCR project/Ground Truth/page_27.txt', "r")
page_27 = (a.read())

a = open('/Users/dhirajhasija/Downloads/OCR project/Ground Truth/page_26.txt', "r")
page_26 = (a.read())

a = open('/Users/dhirajhasija/Downloads/OCR project/Ground Truth/page_24.txt', "r")
page_24 = (a.read())

a = open('/Users/dhirajhasija/Downloads/OCR project/Ground Truth/page_25.txt', "r")
page_25 = (a.read())

a = open('/Users/dhirajhasija/Downloads/OCR project/Ground Truth/page_23.txt', "r")
page_23 = (a.read())

a = open('/Users/dhirajhasija/Downloads/OCR project/Ground Truth/page_22.txt', "r")
page_22 = (a.read())

a = open('/Users/dhirajhasija/Downloads/OCR project/Ground Truth/page_21.txt', "r")
page_21 = (a.read())

In [ ]:
paytm = 'Paytm, Payment, Successful, ₹, 3,002, Paid, at, Jassowal, HP, Centre, BHADSON, ROAD, PATIALA, From, QR, 07, Nov, 2021,, 04:43:48, PM, Payment, Details, Txn, ID, 202111071112128001101680, 66761785546, Order, ID, 20211107164346004724, 10387082, Card, No., Transaction, Type, SALE, Serial, No., 1490593894, MID, Jassow20767837367485, TID, 10387082, Customer, Copy, PAYTM, POS, Version, 1.0.0.0'
HP = 'G-50, HP, HDFC, BANK, JASSOWAL,, HP, CENTRE, JASSOWAL, BHADSON, ROAD, PATIALA, M:9876317400, A09/2021, ORIGINAL, 07-NOV-2021, 17:07:50, TXN, NO:, 1110708612, INVOICE, NO:, 652744, VEHICLE, NO:, NOT, ENTERED, BANK, PRESET:, NOT, ENTERED, HDFC, NOZZLE, NO, :, 1, PRODUCT:, PETROL, RATE, :, 105.26, INR/Ltr, VOLUMNE:, 28.52, Ltr, AMOUNT:, 3002.01, INR, 5001, Thank, You!, VIS'
gvr = 'A110DEE, ESS, ENTERPRICES, PATIALA, IOCL, DEALER, Bill, No:Jan-875537-ORGNL, Trns.ID:0000000300644811, Atnd.ID:, Receipt:Physical, Receipt, Vehi.No:2, Mob.No, :NotEntered, Date, :19/01/2022, Time, :13:48:55, FP., ID, :2, Nozl, No, :, 2, Fuel, :PETROL, Preset, :NON, PRESET, Rate, :Rs.95.41, Sale, :Rs.2898.55, Volume, :30.38L, THANKS, VISIT, AGAIN, GVR'
qrcode = 'more, ., Discount, Supermarket, More, Retail, Private, Limited, (, Formerly, Knows, as, More, Retail, Limited, ), CIN, No., :, U65990MH1988PTC048117, Bhadson, Road, ,, Patiala, JP, Propertes, ,, Near, State, Bank, of, Patiala, ,, Punjab, Tel, :, +917717303803, GSTIN, :, 03AAACP2678Q1ZO, FSSAI, License, No, :, 12115681000018, TAX, Invoice, Date, :, 17, /, 11, /, 21, 19, :, 04, :, 47, CM, No, .:, 1806-4143622422, Staff, :, Talib, Trans, :, 621520, Ref, ., No, :, POS, No, :, J4143, Description, HSN, Qty, Unit, Rate, Amount, 1, SGST, @, 2.5, %, CGST, @, 2.5, %, Borges, Extra, Light, Olive, Oil, 2, Ltr, Pet, Bottle, 15099010, 1ea, 1249, 1249, Jivo, Canola, Oil, 1, Ltr, Pet, Bottle, 15141920, 2ea, 255, 510, 2, SGST, @, 6, %, CGST, @, 6, %, Mothers, Recipe, Schezwan, Chutney, 250, gm, Bottle, 21039090, 1ea, 80, 80, 3, SGST, @, 9, %, CGST, @, 9, %, Vega, Paddle, Brush, E11, -, PB, 96032900, 1ea, 269.1, 269.1, Items, :, 4, Qty, :, 5, Amt, :, 2,108.10, Payment, Summary, ICICI, Bank, EDC,2,108.10, Appr, Code, 8520, *, YOUR, CLUBMORE, SAVINGS,*, 1,620.90, CLUBMORE, Membership, No.,:,GST, Reciept, Summary, Sr, Taxable, Amt, ., 1, /, SGST, CGST, CESS, &, Adv, Total, Amt., 1, 1,675.23, 2, 71.43, 3, 228.05, 41.88, 41.88, 0.00, 1,759.00, 4.29, 4.29, 0.00, 80.00, 20.52, 20.52, 0.00, 269.10, T, 1,974.71, 66.69, 66.69, 0.00, 2,108.10'
Invoice_4 = 'August, House, Cafe, Adinath, Cafe, LLP, GSTIN, :, 08ABPFA1865M1ZV, Name:, Date:, 22/03/22, Dine, In:, O3, 19:06, Cashier:, biller, Bill, No.:, 1145, Persons:, 1, No.Item, Qty., Price, Amount, 1, Iced, Americano, 1, 150.00, 150.00, Total, Qty:, 1, Sub, Total, 150.00. Service, Charge, 7.50, CGST, 2.5%, 3.75,, SGST, 2.5%, 3.75, Grand, Total, 165.00, FSSAI, Lic, No., 12219026001304, Thanks'
Invoice_2 = "HP,ADHOC,RAM,FILLING,STN,VILL.,RAIMALIPUR.,DIST,.M/GARH.HARYANA.,GST.,06AFQPA2094F1ZL,MOB.,9783960007.,Bill,No:95954-ORGNL,Trns.,ID:,Atnd.,ID:,Receipt:,Physical,Receipt,Vehi.,,No:7169,Mob.,No,:,NotEntered,Date,:, 07/03/2022,Time,:,15:01:12,FP.,ID,:,2,Nozl,No:2,Fuel,:,Density,:,830kg/m3,Preset,:,NON,PRESET,Rate,:,Rs.,87.26,Sale,:,Rs.,5835.07,Volume,:,66.87Lts."
Invoice_3 = 'Half, Light, Coffee, Roasters, F-58, ,Sunder, Marg, , C, Scheme, Ph., 9928278748, Email:, halflightcoffee@gmail.com, GSTIN:, 08AVWPG4893Q1ZA, Type:, Take, Away, Bill, No., :119919, Delivery, Boy:, Date:2022-03-21, 16:32:20, Kots:122, Item, Qty, Amt, Iced, Americano, 1, 133.33, Total, Qty:, 1, SubTotal, :, 133.33, GST, 6.67, CGST, @2.5, 3.33, SGST, @2.5, 3.33, Round, Off:, 0.00, Total, Invoice, Value, 140, Thank, you, , visit, again!, Powered, by, -, POSIST '

In [ ]:
# Calculatring jaccard
def jaccard(string, folder):
    list1 = string
    list2 = ''
    if folder == '21':
        list2 = page_21
    
    elif folder == '22':
        list2 = page_22
        
    elif folder == '23':
        list2 = page_23
    
    elif folder == '24':
        list2 = page_24
        
    elif folder == '25':
        list2 = page_25
        
    elif folder == '26':
        list2 = page_26
        
    elif folder == '27':
        list2 = page_27
        
    elif folder == '28':
        list2 = page_28
        
    elif folder == '29':
        list2 = page_29
        
    elif folder == '30':
        list2 = page_30

    elif folder == 'paytm':
        list2 = paytm

    elif folder == 'HP':
        list2 = HP

    elif folder == 'gvr':
        list2 = gvr

    elif folder == 'Invoice_2':
        list2 = Invoice_2

    elif folder == 'Invoice_3':
        list2 = Invoice_3

    elif folder == 'Invoice_4':
        list2 = Invoice_4

    
    list1 = re.split(r'\s+|[,:/!=\'€"_()\[\];\.-]\s*', list1)
    list2 = re.split(r'\s+|[,:/!=\'€"_()\[\];\.-]\s*', list2)
    
    list1 = [i for i in list1 if i]
    list2 = [i for i in list2 if i]
    
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [ ]:
#Calculating cosine sim
def cos_sim(string, folder):
        
    cos_simm = np.zeros((2,2))
    
    if folder == '21':
        list2 = page_21
    
    elif folder == '22':
        list2 = page_22
        
    elif folder == '23':
        list2 = page_23
    
    elif folder == '24':
        list2 = page_24
        
    elif folder == '25':
        list2 = page_25
        
    elif folder == '26':
        list2 = page_26
        
    elif folder == '27':
        list2 = page_27
        
    elif folder == '28':
        list2 = page_28
        
    elif folder == '29':
        list2 = page_29
        
    elif folder == '30':
        list2 = page_30
        
    elif folder == 'paytm':
        list2 = paytm
    
    elif folder == 'HP':
        list2 = HP
        
    elif folder == 'gvr':
        list2 = gvr
        
    elif folder == 'Invoice_2':
        list2 = Invoice_2
        
    elif folder == 'Invoice_3':
        list2 = Invoice_3
        
    elif folder == 'Invoice_4':
        list2 = Invoice_4
        

    document = [list2,string]
    document = count_vectorizer.fit_transform(document)
    cos_simm = cosine_similarity(document, document)

    
    return (cos_simm[1][0])

In [ ]:
#Calculating cosine sim
def euc_dist(string, folder):
        
    cos_simm = np.zeros((2,2))
    
    if folder == '21':
        list2 = page_21
    
    elif folder == '22':
        list2 = page_22
        
    elif folder == '23':
        list2 = page_23
    
    elif folder == '24':
        list2 = page_24
        
    elif folder == '25':
        list2 = page_25
        
    elif folder == '26':
        list2 = page_26
        
    elif folder == '27':
        list2 = page_27
        
    elif folder == '28':
        list2 = page_28
        
    elif folder == '29':
        list2 = page_29
        
    elif folder == '30':
        list2 = page_30
        
    elif folder == 'paytm':
        list2 = paytm
    
    elif folder == 'HP':
        list2 = HP
        
    elif folder == 'gvr':
        list2 = gvr
        
    elif folder == 'Invoice_2':
        list2 = Invoice_2
        
    elif folder == 'Invoice_3':
        list2 = Invoice_3
        
    elif folder == 'Invoice_4':
        list2 = Invoice_4
        

    document = [list2,string]
    document = count_vectorizer.fit_transform(document)
    cos_simm = euclidean_distances(document, document)

    
    return (cos_simm[1][0])

In [ ]:
def CERT(string, folder):
    
    st1 = string
    st2 = ''
    list2 = ''
    
    if folder == '21':
        list2 = page_21
    
    elif folder == '22':
        list2 = page_22
        
    elif folder == '23':
        list2 = page_23
    
    elif folder == '24':
        list2 = page_24
        
    elif folder == '25':
        list2 = page_25
        
    elif folder == '26':
        list2 = page_26
        
    elif folder == '27':
        list2 = page_27
        
    elif folder == '28':
        list2 = page_28
        
    elif folder == '29':
        list2 = page_29
        
    elif folder == '30':
        list2 = page_30

    elif folder == 'paytm':
        list2 = paytm
    
    elif folder == 'HP':
        list2 = HP
        
    elif folder == 'gvr':
        list2 = gvr
        
    elif folder == 'Invoice_2':
        list2 = Invoice_2
        
    elif folder == 'Invoice_3':
        list2 = Invoice_3
        
    elif folder == 'Invoice_4':
        list2 = Invoice_4
        
    st2 = list2
    
    list1 = re.split(r'\s+|[\n,:/!=\'€"_()\[\];\.-]\s*', st1)
    list2 = re.split(r'\s+|[\n,:/!=\'€"_()\[\];\.-]\s*', st2)
    

    list1 = [i for i in list1 if i]
    list2 = [i for i in list2 if i]
    
    st1 =  ' '.join(np.sort(list1))
    st2 =  ' '.join(np.sort(list2))
    
    CER = fastwer.score_sent(st1, st2, char_level=True)
    
    return CER

In [ ]:
df_comparison['cos_ggl'] = df_comparison.apply(lambda x: cos_sim((' '.join(x['words'])),str(x['Doc_no'])), axis =1)
df_comparison['euc_ggl'] = df_comparison.apply(lambda x: euc_dist(' '.join((x['words'])),str(x['Doc_no'])), axis =1)
df_comparison['Jac_ggl'] = df_comparison.apply(lambda x: jaccard((' '.join(x['words'])),str(x['Doc_no'])), axis =1)

# Notes

1. Amazon's Dataframe has doc no as D21 and our google's dataframe has 21
2. Amazon's DF Doesn't have jpg in image name; GGl's DF has
2. Amazon's Dataframe has some additional column

## Next code might need modification based on Amazon's OCR's Dataframe

# Phase 2 : Calculating metrics

In [ ]:
df = pd.read_csv('Phase 2/df_Textract_All dataset_24_6.csv')

In [ ]:
df.columns

Index(['Img_name', 'Response_json', 'Json_path', 'Img_path', 'N_bbox',
       'Extracted_text', 'Mean_conf_score', 'Med_conf_score', 'conf_array',
       'Ground_truth', 'Dataset_name', 'API', 'Base_path', 'Img_no'],
      dtype='object')

In [ ]:
df[['Img_name','Dataset_name']].groupby(by = df['Dataset_name']).head(5)

,Img_name,Dataset_name
0,Fontfrm_Noisec_RE_gs,Noisy
1,FontLtm_Noisec_RE_gs,Noisy
2,FontLse_Noisew_RE_gs,Noisy
3,Fontnse_Noisep_RE_gs,Noisy
4,Fontnsm_Noisep_RE_gs,Noisy
144,M_Img_WP_D24_L2_r35_a0_b0,SmartDocQA
145,M_Img_WP_D24_L3_r35_a-5_b10,SmartDocQA
146,M_Img_WP_D24_L4_r35_a0_b0,SmartDocQA
147,M_Img_WP_D24_L3_r35_a0_b0,SmartDocQA
148,M_Img_WP_D24_L2_r35_a-5_b10,SmartDocQA


In [ ]:
df['Ground_truth'] = df['Ground_truth'].to_string()
df['Extracted_text'] = df['Extracted_text'].to_string()

In [ ]:
def cleaning_text(text):
    list1 = re.split(r'\s+|[,:/!=\'€"_()\[\];\.-]\s*', text)
    list1 = [i for i in list1 if i]
    text = ' '.join(list1)
    text = text.lower()
    return text

df['Extracted_text_cleaned'] = df['Extracted_text'].apply(lambda x:cleaning_text(x))
df['Ground_truth_cleaned'] = df['Ground_truth'].apply(lambda x:cleaning_text(x))

In [ ]:
def CER(original, extracted):
    CER = fastwer.score_sent(extracted, original, char_level=True)
    return CER

df['CER'] = df.apply(lambda x : CER(x['Ground_truth_cleaned'], x['Extracted_text_cleaned']), axis = 1)

In [ ]:
def WER(original, extracted):
    WER = fastwer.score_sent(extracted, original)
    return WER

df['WER'] = df.apply(lambda x : WER(x['Ground_truth_cleaned'], x['Extracted_text_cleaned']), axis = 1)

In [ ]:
def Jaccard(original, extracted):
    list1 = original.split()
    list2 = extracted.split()
    
    intersection = len(list(set(list1).intersection(set(list2))))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection / union)

df['Jaccard Index'] = df.apply(lambda x : Jaccard(x['Ground_truth_cleaned'], x['Extracted_text_cleaned']), axis = 1)

In [ ]:
count_vectorizer = CountVectorizer(stop_words='english')
count_vectorizer = CountVectorizer()

def cos(original, extracted):
    
    document = [original, extracted]
    document = count_vectorizer.fit_transform(document)
    cos_simm = cosine_similarity(document, document)

    
    return (cos_simm[1][0])

df['Cosine Metrics'] = df.apply(lambda x : cos(x['Ground_truth_cleaned'], x['Extracted_text_cleaned']), axis = 1)

In [ ]:
def word_ratio(original, extracted):
    l1 = len(original.split())
    l2 = len(extracted.split())
    return (l2/l1)

df['N_word_ratio'] = df.apply(lambda x : word_ratio(x['Ground_truth_cleaned'], x['Extracted_text_cleaned']), axis = 1)

# Doubts
1. N_word_ratio is greater than 1 also.
2. how to split on \ in regex, use of \s* at last in regex